# Book Example: Economic equilibria
[![econ_lecture.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/ampl-lecture/econ_lecture.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/ampl-lecture/econ_lecture.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/ampl-lecture/econ_lecture.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/ampl-lecture/econ_lecture.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/ampl-lecture/econ_lecture.ipynb)

Description: economic model using complementarity conditions from Chapter 19 AMPL book

Tags: ampl-only, ampl-book, complementarity_problem

Notebook author: N/A

Model author: N/A


In [1]:
# Install dependencies
!pip install -q amplpy ampltools

In [2]:
# Google Colab & Kaggle interagration
MODULES=['ampl', 'coin', 'x-gurobi']
from ampltools import cloud_platform_name, ampl_notebook
from amplpy import AMPL, register_magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = ampl_notebook(modules=MODULES, reinstall=True) # Install AMPL and use it
register_magics(ampl_object=ampl) # Evaluate %%ampl_eval cells with ampl.eval()

Downloading: https://portal.ampl.com/dl/modules/ampl-module.linux64.tgz
> /content/ampl.linux-intel64/leasefingerprint (replacing)
> /content/ampl.linux-intel64/fingerprint (replacing)
> /content/ampl.linux-intel64/amplkey (replacing)
> /content/ampl.linux-intel64/eodbc.dll (replacing)
> /content/ampl.linux-intel64/ampl.lic (replacing)
> /content/ampl.linux-intel64/docs (replacing)
> /content/ampl.linux-intel64/models (replacing)
> /content/ampl.linux-intel64/ampl.env (replacing)
> /content/ampl.linux-intel64/amplcsv.dll (replacing)
> /content/ampl.linux-intel64/amplxl.dll (replacing)
> /content/ampl.linux-intel64/README (replacing)
> /content/ampl.linux-intel64/ampl (replacing)
Downloading: https://portal.ampl.com/dl/modules/coin-module.linux64.tgz
> /content/ampl.linux-intel64/ipopt (replacing)
> /content/ampl.linux-intel64/couenne (replacing)
> /content/ampl.linux-intel64/coin-versions.txt (replacing)
> /content/ampl.linux-intel64/coin-license.txt (replacing)
> /content/ampl.linux-i

## Economic equilibria
This model is based on the models showed in the Chapter 19 of the AMPL book, related to **complementarity problems**.

### Classic model
The following model does not use complementarity conditions.

* Sets and parameters:
     * Products `PROD` and `demand[i]` for each product `i`.
     * Activities `ACT` and `cost[j]` for each activity `j`.
     * Amount of product `i` produced by activity `j`, `io[i,j]`.
* Decision variables `Level[j]`: levels of production activities (for each activity `j`).
* Objective: minimize total production cost,
$$\sum \limits_{j \in ACT} cost_j \cdot Level_j;$$

So the AMPL formulation is implemented as:

In [3]:
%%writefile econmin.mod
set PROD;   # products
set ACT;    # activities

param cost {ACT} > 0;      # cost per unit of each activity
param demand {PROD} >= 0;  # units of demand for each product
param io {PROD,ACT} >= 0;  # units of each product from
                           # 1 unit of each activity

var Level {j in ACT} >= 0;

minimize Total_Cost:  sum {j in ACT} cost[j] * Level[j];

subject to Demand {i in PROD}:
      sum {j in ACT} io[i,j] * Level[j] >= demand[i];

Overwriting econmin.mod


In [4]:
%%writefile econ.dat
data;

param: ACT:   cost :=
        P1    2450
        P1a   1290
        P2    1850
        P2a   3700
        P2b   2150
        P3    2200
        P3c   2370
        P4    2170 ;

param: PROD:  demand :=
       AA1     70000 
       AC1     80000
       BC1     90000 
       BC2     70000 
       NA2    400000 
       NA3    800000  ;

param io (tr):
         AA1  AC1  BC1  BC2   NA2   NA3 :=
   P1     60   20   10   15   938   295
   P1a     8    0   20   20  1180   770
   P2      8   10   15   10   945   440
   P2a    40   40   35   10   278   430
   P2b    15   35   15   15  1182   315
   P3     70   30   15   15   896   400
   P3c    25   40   30   30  1029   370
   P4     60   20   15   10  1397   450 ;


Overwriting econ.dat


In [5]:
%%ampl_eval
model econmin.mod;
data econ.dat;
option solver cbc;
solve;
display Level;
display Demand.dual;

CBC 2.10.5: CBC 2.10.5 optimal, objective 6808640.553
3 iterations
Level [*] :=
 P1     0
P1a  1555.3
 P2     0
P2a     0
P2b     0
 P3   147.465
P3c  1889.4
 P4     0
;

Demand.dual [*] :=
AA1  16.7051
AC1   5.44585
BC1  57.818
BC2   0
NA2   0
NA3   0
;



### Complementarity model

Consider the new variables `Price[i]` for each product, and solve the problem to find an *equilibrium* instead of an optimum solution. The equilibrium is subject to two conditions:

* First, for each product, total output must meet demand and price must be nonnegative, and in addition there must be a *complementarity* between these relationships, where production exceeds demand the price must be zero, or equivalently, if the price is positive, the production must equal the demand.

```
subject to Pri_Compl {i in PROD}:
   Price[i] >= 0 complements
      sum {j in ACT} io[i,j] * Level[j] >= demand[i];
```

* Second, for each activity `j`, the value of resulting product `i` is `Price[i]*io[i,j]`, so activity `j` would produce a total value of 

$$\sum \limits_{i \in PROD} Price[i] \cdot io[i,j]$$

When equilibrium happens, the previous value must not exceed the activity's cost per unit `cost[j]`. Moreover, there is a complementarity between this relationship and the level of activity `j`, where cost exceeds total value the activity must be zero, or, where the activity cost is positive then the total value must be equal to the cost. This can be expressed as:

```
subject to Lev_Compl {j in ACT}:
   Level[j] >= 0 complements
      sum {i in PROD} Price[i] * io[i,j] <= cost[j];
```

In [6]:
%%writefile econ.mod
set PROD;   # products
set ACT;    # activities

param cost {ACT} > 0;      # cost per unit of each activity
param demand {PROD} >= 0;  # units of demand for each product
param io {PROD,ACT} >= 0;  # units of each product from
                           # 1 unit of each activity
var Price {i in PROD};
var Level {j in ACT};

subject to Pri_Compl {i in PROD}:
   Price[i] >= 0 complements
      sum {j in ACT} io[i,j] * Level[j] >= demand[i];

subject to Lev_Compl {j in ACT}:
   Level[j] >= 0 complements
      sum {i in PROD} Price[i] * io[i,j] <= cost[j];

Overwriting econ.mod


**Remark**: the model is square, as there are $n+m$ variables and $n+m$ constraints ($n$ number of products and $m$ number of activities). Some solvers can take advantage of this to use different solving techniques.

Finally, let's see that the equilibrium model gives also the optimal solution for the first classic formulation.

In [7]:
%%ampl_eval
reset;
model econ.mod;
data econ.dat;
option solver x-gurobi;
solve;
# Show total cost
display sum {j in ACT} cost[j] * Level[j];
display Level;

x-Gurobi 9.5.1: x-Gurobi 9.5.1: optimal solution; objective 0
sum{j in ACT} cost[j]*Level[j] = 6808640

Level [*] :=
 P1     0
P1a  1555.3
 P2     0
P2a     0
P2b     0
 P3   147.465
P3c  1889.4
 P4     0
;

